# Module 01: Stock Market Essentials

## Goal
Learn to load financial data, understand OHLCV format, and create professional visualizations using Plotly.

## Setup

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 1. Load Data
We use the sample NIFTY 50 (or proxy) data provided in `demo_data/`.

In [ ]:
# Load CSV
df = pd.read_csv('../demo_data/nifty50_sample.csv')

# Convert Date column to datetime objects
df['Date'] = pd.to_datetime(df['Date'])

# Set Date as index for easier time-series manipulation
df.set_index('Date', inplace=True)

# Create a 'Close' copy for calculations
df.sort_index(inplace=True)

print("Data Shape:", df.shape)
df.head()

## 2. Basic Visualization (Line Chart)
A line chart connects the closing prices.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Close Price'))
fig.update_layout(title='Stock Price History (Line Chart)', xaxis_title='Date', yaxis_title='Price')
fig.show()

## 3. Candlestick Chart
Candlesticks show Open, High, Low, and Close prices for each period.

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.update_layout(title='Stock Price (Candlestick)', xaxis_rangeslider_visible=False)
fig.show()

## 4. Moving Average (SMA)
Simple Moving Average smoothes out price data to identify trends.
$$ SMA_n = \frac{P_1 + P_2 + ... + P_n}{n} $$

In [ ]:
# Calculate 20-day SMA
df['SMA_20'] = df['Close'].rolling(window=20).mean()

# Plot Candlestick + SMA
fig = go.Figure()

# Add Candlestick
fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='OHLC'))

# Add SMA Line
fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], line=dict(color='orange', width=2), name='SMA 20'))

fig.update_layout(title='Candlestick with 20-Day SMA', yaxis_title='Price', xaxis_rangeslider_visible=False)
fig.show()

## 5. Calculate Daily Returns
Log returns are preferred for statistical analysis.

In [ ]:
# Simple Returns: (Price_t / Price_{t-1}) - 1
df['Simple_Return'] = df['Close'].pct_change()

# Log Returns: ln(Price_t / Price_{t-1})
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

# Drop NaN values created by shift
df.dropna(inplace=True)

print(df[['Close', 'Simple_Return', 'Log_Return']].head())

# Plot Histogram of Returns
fig = go.Figure(data=[go.Histogram(x=df['Log_Return'], nbinsx=50)])
fig.update_layout(title='Distribution of Daily Log Returns')
fig.show()